In [1]:
import pandas as pd

In [2]:
def save_csv_for_mathematica(df: pd.DataFrame, name: str):
    df.to_csv(f"data_for_mathematica/{name}.csv")

In [3]:
enefit = pd.read_csv("EnefitAPI/train.csv")
enefit["datetime"] = pd.to_datetime(enefit["datetime"])

consumption = enefit[enefit["is_consumption"]==1]
monthlyCons = consumption.groupby(pd.Grouper(key="datetime", freq='M')).mean() 
weeklyCons = consumption.groupby(pd.Grouper(key="datetime", freq='W')).mean() 
dailyCons = consumption.groupby(pd.Grouper(key="datetime", freq='D')).mean() 
meanConsumption = consumption.target.mean()

# save_csv_for_mathematica(dailyCons.target, "daily_consumption_train")

In [4]:
enefit

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id
0,0,0,1,0.713,0,2021-09-01 00:00:00,0,0,0
1,0,0,1,96.590,1,2021-09-01 00:00:00,0,1,0
2,0,0,2,0.000,0,2021-09-01 00:00:00,0,2,1
3,0,0,2,17.314,1,2021-09-01 00:00:00,0,3,1
4,0,0,3,2.904,0,2021-09-01 00:00:00,0,4,2
...,...,...,...,...,...,...,...,...,...
2018347,15,1,0,197.233,1,2023-05-31 23:00:00,637,2018347,64
2018348,15,1,1,0.000,0,2023-05-31 23:00:00,637,2018348,59
2018349,15,1,1,28.404,1,2023-05-31 23:00:00,637,2018349,59
2018350,15,1,3,0.000,0,2023-05-31 23:00:00,637,2018350,60


In [5]:
production = enefit[enefit["is_consumption"]==0]
dailyProd = production.groupby(pd.Grouper(key="datetime", freq='D')).mean() 
meanProduction = production.target.mean()

# save_csv_for_mathematica(dailyProd.target, "daily_production_train")

In [6]:
business = enefit.query("is_business == 1 & is_consumption == 1")
businessDailyCons = business.groupby(pd.Grouper(key="datetime", freq="D")).mean()
businessDailyCons.target.mean()

# save_csv_for_mathematica(businessDailyCons.target, "business_consumption_train")

745.3075264179948

In [7]:
weather = pd.read_csv("../../EnefitBigData/historical_weather.csv")
weather["datetime"] = pd.to_datetime(weather["datetime"])
temperatureValues = weather.groupby(pd.Grouper(key="datetime", freq="D")).mean()
temperatureValues.temperature.mean()

# save_csv_for_mathematica(temperatureValues.temperature, "temperature")

5.707557127184041

In [8]:
gas = pd.read_csv("EnefitAPI/gas_prices.csv")
gas.drop(["origin_date"], inplace=True, axis=1)
gas["forecast_date"] = pd.to_datetime(gas["forecast_date"])
gasValues = gas.groupby(pd.Grouper(key="forecast_date", freq="D")).mean()
gasValues = gasValues[["lowest_price_per_mwh","highest_price_per_mwh"]]
print("Lowest: " + str(gasValues.lowest_price_per_mwh.mean()))
print("Highest: " + str(gasValues.highest_price_per_mwh.mean()))
gasValues

# save_csv_for_mathematica(gasValues, "Lowest_Highest_price_per_mwh")

Lowest: 95.03675039246468
Highest: 107.75463108320253


,lowest_price_per_mwh,highest_price_per_mwh
forecast_date,,
2021-09-01,45.23,46.32
2021-09-02,45.62,46.29
2021-09-03,45.85,46.40
2021-09-04,46.30,46.80
2021-09-05,46.30,46.58
...,...,...
2023-05-26,29.10,34.10
2023-05-27,28.30,34.10
2023-05-28,28.10,34.10


In [9]:
energy = pd.read_csv("EnefitAPI/electricity_prices.csv")
energy.drop(["origin_date"], inplace=True, axis=1)
energy["forecast_date"] = pd.to_datetime(energy["forecast_date"])
energyValues = energy.groupby(pd.Grouper(key="forecast_date", freq="D")).mean()
energyValues = energyValues[["euros_per_mwh"]]
print("Avg: " + str(energyValues.euros_per_mwh.mean()))

# save_csv_for_mathematica(energyValues, "daily_energy_price_per_mwh")

Avg: 157.05207101904304


In [30]:
# normalized_df=(df-df.min())/(df.max()-df.min())
weather_to_normalize = weather.groupby(['latitude', 'longitude'])[['direct_solar_radiation', 'cloudcover_low']].mean().reset_index()
SR = weather_to_normalize['direct_solar_radiation']
CC = weather_to_normalize['cloudcover_low']
weather_to_normalize['direct_solar_radiation'] = (SR-SR.min())/(SR.max()-SR.min())
weather_to_normalize['cloudcover_low'] = (CC-CC.min())/(CC.max()-CC.min())
weather_to_normalize['diff'] = weather_to_normalize['direct_solar_radiation'] - weather_to_normalize['cloudcover_low']
save_csv_for_mathematica(weather_to_normalize.drop(['direct_solar_radiation', 'cloudcover_low'], axis=1).set_index('longitude'), 'optimal_place')
